In [36]:
from google.oauth2 import service_account
from googleapiclient.discovery import build

SCOPES = ['https://www.googleapis.com/auth/drive']
SERVICE_ACCOUNT_FILE = 'resume-mdl-209aeff2da09.json'
DOCUMENT_ID = '1N7AOH4gO5dyG6uqh_KWYeyN42vX36UMCCiOlXFbgYh4'

In [32]:
def get_credentials():
    credentials = service_account.Credentials.from_service_account_file(
        SERVICE_ACCOUNT_FILE, scopes=SCOPES)

    delegated_credentials = credentials.with_subject('caleb@calebeverett.io')
    
    return delegated_credentials
    
def read_paragraph_element(element):
    """Returns the text in the given ParagraphElement.

        Args:
            element: a ParagraphElement from a Google Doc.
    """
    text_run = element.get('textRun')
    if not text_run:
        return ''
    return text_run.get('content')


def read_strucutural_elements(elements):
    """Recurses through a list of Structural Elements to read a document's text where text may be
        in nested elements.

        Args:
            elements: a list of Structural Elements.
    """
    text = ''
    for value in elements:
        if 'paragraph' in value:
            elements = value.get('paragraph').get('elements')
            for elem in elements:
                text += read_paragraph_element(elem)
        elif 'table' in value:
            # The text in table cells are in nested Structural Elements and tables may be
            # nested.
            table = value.get('table')
            for row in table.get('tableRows'):
                cells = row.get('tableCells')
                for cell in cells:
                    text += read_strucutural_elements(cell.get('content'))
        elif 'tableOfContents' in value:
            # The text in the TOC is also in a Structural Element.
            toc = value.get('tableOfContents')
            text += read_strucutural_elements(toc.get('content'))
    return text

In [33]:
def main():
    credentials = get_credentials()
    service = build('docs', 'v1', credentials=credentials)
    doc = service.documents().get(documentId=DOCUMENT_ID).execute()
    doc_content = dot.get('body').get('content')
    with open("data/welcome.toml", "w") as text_file:
        text_file.write(read_strucutural_elements(doc_content))

In [34]:
if __name__ == '__main__':
    main()